In [1]:
from selenium import webdriver  
import time  
import urllib  
from slugify import slugify
import requests
import random
import os


In [38]:
conference_url = "https://openreview.net/group?id=NeurIPS.cc/2021/Conference" # the conference url to download papers from
Edgedriver_path = 'C:\\Users\\xiaoyang\\Downloads\\edgedriver_win64\msedgedriver.exe' # the edgedriver.exe path
root = r'E:\OneDrive\academics\papers\conferences\NeurIPS-2021-ALL'.replace('\\','/') # file path to save the downloaded papers


In [170]:
os.makedirs(root, exist_ok=True)
print(root)

driver = webdriver.Edge(Edgedriver_path)  
driver.get(conference_url)


E:/OneDrive/academics/papers/conferences/NEURIPS-2021-ALL


In [171]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

js = "window.scrollTo(0,0)"

pdfurllist =  []
pdfnamelist = []

# three sections: oral, spotlight, poster
for num_section, section in enumerate(['Oral', 'Spotlight', 'Poster']):
    # my_action.move_to_element(driver.find_elemen_by_partial_link_text(section)).perform()
    driver.execute_script(js)
    driver.find_element_by_partial_link_text(section).click()
    time.sleep(3)
    elementllist = driver.find_elements_by_xpath('//*[@id="{}-presentations"]/ul/li/h4'.format(section.lower()))
    for i, element in enumerate(elementllist): 
        pdfnamelist.append(elementllist[i].text)
        pdfurllist.append(elementllist[i].find_elements_by_xpath('a')[1].get_attribute('href'))
    next_page = True
    while next_page:
        try:
            driver.find_elements_by_class_name('right-arrow')[num_section*2].find_element_by_xpath('a').click()
            time.sleep(1)
            elementllist = driver.find_elements_by_xpath('//*[@id="{}-presentations"]/ul/li/h4'.format(section.lower()))
            for i, element in enumerate(elementllist): 
                pdfnamelist.append(elementllist[i].text)
                pdfurllist.append(elementllist[i].find_elements_by_xpath('a')[1].get_attribute('href'))
            print(len(pdfnamelist))
        except ElementClickInterceptedException as e:
            time.sleep(1)
        except NoSuchElementException as e:
            print(section, 'no more pages', len(pdfnamelist))
            next_page = False
    # print(len(pdfnamelist))

50
60
Oral no more pages 60
110
135
160
185
210
235
260
285
310
335
345
Spotlight no more pages 345
395
420
445
470
495
520
545
570
595
620
645
670
695
720
745
770
795
820
845
870
895
920
945
970
995
1020
1045
1070
1095
1120
1145
1170
1195
1220
1245
1270
1295
1320
1345
1370
1395
1420
1445
1470
1495
1520
1545
1570
1595
1620
1645
1670
1695
1720
1745
1770
1795
1820
1845
1870
1895
1920
1945
1970
1995
2020
2045
2070
2095
2120
2145
2170
2195
2220
2245
2270
2295
2320
2345
2370
2395
2420
2445
2470
2495
2520
2545
2570
2595
2620
2632
Poster no more pages 2632


In [172]:

# check the retrieved urls
print('The first 5 pdf urls:\n')
for i in range(5):
    print(pdfurllist[i])
print('\nThe last 5 pdf urls:\n')
for i in range(5):
    print(pdfurllist[-(i+1)])
print('=======================================================')


The first 5 pdf urls:

https://openreview.net/pdf?id=IVV1putQ90
https://openreview.net/pdf?id=an8FSGbuCw
https://openreview.net/pdf?id=AzmEMstdf3o
https://openreview.net/pdf?id=GlEWs-V9boR
https://openreview.net/pdf?id=kwN2xvZ2XZ9

The last 5 pdf urls:

https://openreview.net/pdf?id=L8-54wkift
https://openreview.net/pdf?id=y2p9IIXwdg2
https://openreview.net/pdf?id=XI72RT3hnnF
https://openreview.net/pdf?id=SAPEODcpNvf
https://openreview.net/pdf?id=Rk7B9kmp7R8


In [173]:
# check the retrieved paper titles
print('The first 5 pdf titles:\n')
for i in range(5):
    print(pdfnamelist[i])
print('\nThe last 5 pdf titles:\n')
for i in range(5):
    print(pdfnamelist[-(i+1)])

The first 5 pdf titles:

Latent Equilibrium: A unified learning theory for arbitrarily fast computation with arbitrarily slow neurons
Latent Equilibrium: A unified learning theory for arbitrarily fast computation with arbitrarily slow neurons
Passive attention in artificial neural networks predicts human visual selectivity
Volume Rendering of Neural Implicit Surfaces
Learning Frequency Domain Approximation for Binary Neural Networks

The last 5 pdf titles:

Lower Bounds and Optimal Algorithms for Smooth and Strongly Convex Decentralized Optimization Over Time-Varying Networks
Deconvolutional Networks on Graph Data
Global Convergence of Online Optimization for Nonlinear Model Predictive Control
RED : Looking for Redundancies for Data-FreeStructured Compression of Deep Neural Networks
Minimax Regret for Stochastic Shortest Path


In [174]:
print('The number of papers is ', len(pdfnamelist))
assert len(pdfnamelist)==len(pdfurllist), 'The number of titles and the number of urls are not matched. \
                                            You might solve the problem by checking the HTML code in the \
                                            website yourself or you could ask the author by raising an issue.'



The number of papers is  2632


In [177]:

# download the papers one by one. The files are named after the titles (guaranteed to be valid file name after processed by slugify.)
print('Start downloading')
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:57.0) Gecko/20100101 Firefox/57.0'}
for i, url in enumerate(pdfurllist):
    if url != None :      
        pdfname = slugify(pdfnamelist[i])
        if i<60:
            sub_directory = 'oral'
        elif i<60+285:
            sub_directory = 'spotlight'
        else:
            sub_directory = 'poster'
        if os.path.isfile(root+'/'+sub_directory+'/'+pdfname+".pdf"):
            print('existed', i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
        else:
            print(i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
            
            try_download = True
            while try_download:
                try:
                    data = requests.get(pdfurllist[i], timeout=80, headers=headers).content
                    try_download = False
                except ConnectionError as e: 
                    _ = time.sleep(random.uniform(1,2)) # for anti-anti-crawler purpose
                    
            with open(root+'/'+sub_directory+'/'+pdfname+".pdf", 'wb')  as f:
                f.write(data)  

Start downloading
existed 0 	 Latent Equilibrium: A unified learning theory for arbitrarily fast computation with arbitrarily slow neurons 	 https://openreview.net/pdf?id=IVV1putQ90
existed 1 	 Latent Equilibrium: A unified learning theory for arbitrarily fast computation with arbitrarily slow neurons 	 https://openreview.net/pdf?id=an8FSGbuCw
existed 2 	 Passive attention in artificial neural networks predicts human visual selectivity 	 https://openreview.net/pdf?id=AzmEMstdf3o
existed 3 	 Volume Rendering of Neural Implicit Surfaces 	 https://openreview.net/pdf?id=GlEWs-V9boR
existed 4 	 Learning Frequency Domain Approximation for Binary Neural Networks 	 https://openreview.net/pdf?id=kwN2xvZ2XZ9
existed 5 	 Learning Frequency Domain Approximation for Binary Neural Networks 	 https://openreview.net/pdf?id=5JvnsAdf6Vz
existed 6 	 MAUVE: Measuring the Gap Between Neural Text and Human Text using Divergence Frontiers 	 https://openreview.net/pdf?id=Tqx7nJp7PR
existed 7 	 On the Expressi